In [1]:
import pandas as pd

In [2]:
asus, lenovo = pd.read_csv('datos/asus/productos-asus.csv'), pd.read_csv('datos/lenovo/productos-lenovo.csv')

In [10]:
df = pd.concat([asus, lenovo]).sort_values(['brand_name', 'title']).reset_index(drop=True)

In [18]:
mask = (df.product_url.isna())
df.loc[mask, 'product_url'] = df.loc[mask].slug + '-' + df.loc[mask].sku_id.astype(str)

In [20]:
df.loc[df.brand_name == 'Lenovo'].product_url.sample().values

array(['auriculares-ear-inalambricos-bluetooth-lenovo-lp11-black-new-version--21262148'],
      dtype=object)

In [23]:
mask = (~df.product_url.str.contains('http'))

df.loc[mask, 'product_url'] = 'https://www.fravega.com/p/' + df.loc[mask].product_url

In [29]:
df = df.sort_values(['brand_name', 'title']).reset_index(drop=True)

In [30]:
df.product_url.sample().values[0]

'https://www.fravega.com/p/lenovo-thinkbook-16-g7-computadora-portatil-22629319'

In [34]:
df.to_csv('datos/database-productos.csv', index=False)